### [9.1 在函数上添加包装器 ](https://python3-cookbook.readthedocs.io/zh_CN/latest/c09/p01_put_wrapper_around_function.html)

这个例子很简单，就是利用函数运行前后的时间差，而 `@wraps` 则是保留函数的 `__doc__`

In [1]:
import time
from functools import wraps

def timethis(func):
    '''
    Decorator that reports the execution time.
    '''
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper

In [2]:
>>> @timethis
... def countdown(n):
...     '''
...     Counts down
...     '''
...     while n > 0:
...         n -= 1

In [5]:
countdown(100000)

countdown 0.00674748420715332
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 7.09 ms


In [6]:
countdown.__doc__

'\n    Counts down\n    '

### [9.4 定义一个带参数的装饰器](https://python3-cookbook.readthedocs.io/zh_CN/latest/c09/p04_define_decorator_that_takes_arguments.html)

In [7]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    """
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    """
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
        return wrapper
    return decorate

In [9]:
# Example use
@logged(logging.DEBUG)
def add(x, y):
    """Return x + y."""
    return x + y

@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [11]:
add(5, 2)

7

In [12]:
spam()

spam


Spam!


In [10]:
add.__doc__

'Return x + y.'

### [9.5 可自定义属性的装饰器](https://python3-cookbook.readthedocs.io/zh_CN/latest/c09/p05_define_decorator_with_user_adjustable_attributes.html)

In [14]:
attach_wrapper('haha')

functools.partial(<function attach_wrapper at 0x7f92a8b13730>, 'haha')

In [15]:
from functools import wraps, partial
import logging
# Utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)  # 这句没怎么看懂
    setattr(obj, func.__name__, func)
    return func

def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__

        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

        # Attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper

    return decorate

In [ ]:
# Example use
@logged(logging.DEBUG)
def add(x, y):
    return x + y

In [13]:
@logged(logging.CRITICAL, 'example')
def spam():
    print('Spam!')

In [16]:
add(1, 2)

3

这个例子太难了，也可能是我不熟悉日志管理的原因